In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import smallgraph as sg
import networkx as nx
from matplotlib import pyplot as plt
import lmz
import eden.graph as eg
import eden.display as ed
from yoda.graphs import ali2graph
import numpy as np
import yoda.ml.simpleMl as sml
import eden.display as ed
from yoda.alignments import load_rfam, filter_by_seqcount

In [ ]:
a,l, te = sg.makedata()

In [ ]:
g =  ali2graph.nearstem( graphs[13].copy(), boost_range = 2, boost_thresh = .5, boost_weight = 100)
ed.draw_graph(g, vertex_color='weight')

In [ ]:
graphs = lmz.Map(ali2graph.writecons, a)
graphs = lmz.Map(ali2graph.dillute, graphs, dilute1 = .7, dilute2 = .4, fix_edges = True)
graphs = lmz.Map(ali2graph.conscut, graphs, 
                 consThresh= .97, replacelabel = False )
# graphs = lmz.Map(ali2graph.nearstem, graphs, boost_range = 3, boost_thresh = .5, boost_weight = 1)

matrix = eg.vectorize(graphs, normalization=True, min_r = 2,min_d=1, inner_normalization=True)
sml.knn_accuracy(matrix,l)

In [ ]:

print([sml.clan_in_x_OLD(matrix,l,n) for n in range(50)])
plt.plot([sml.clan_in_x_OLD(matrix,l,n) for n in range(50)])


In [ ]:
from ubergauss import optimization as op
import yoda.ml.simpleMl as sml

manyseq, fewseq = filter_by_seqcount(a)

def eval(ali,labels,
         ct = .95, 
         rl = False,
         norm = True, 
         d1 = .7,
         d2= .4,
         bad_weight = 0.1,
         fix_edges = True,
         boost_range = 3, boost_thresh = .5, boost_weight = 1,
         min_r = 2, 
         min_d = 1):
    
    graphs = lmz.Map(ali2graph.writecons, ali)
    graphs = lmz.Map(ali2graph.dillute, graphs, dilute1 = d1, dilute2 = d2, fix_edges = fix_edges )
    graphs = lmz.Map(ali2graph.conscut, graphs,  bad_weight = bad_weight, consThresh= ct, replacelabel = rl )
    # graphs = lmz.Map(ali2graph.nearstem, graphs, boost_range = boost_range, boost_thresh = boost_thresh, boost_weight = boost_weight)
    # graphs = [a.graph for a in ali]
    
    matrix = eg.vectorize(graphs, normalization=norm,
                          min_r = min_r,min_d=min_d,
                          inner_normalization=norm) #.toarray()
    
    # ret= sml.knn_accuracy(matrix,labels)
    ret= sml.knn_accuracy(matrix,labels, select_labels=manyseq)
    return ret


space = '''rl 0 1 1
d1 0 1
d2 0 1
bad_weight 0 .5'''
space = sg.string_to_space(space)

z= op.gridsearch(eval, space , tasks = [space.sample() for i in range(40)] , data= [a,l], mp=True)

In [ ]:
eval(a,l,ct = .95, d1 = .7, d2= .4)

In [ ]:
import seaborn as sns
sns.pairplot(z, hue='score')
plt.show()
sg.uo.get_best(z)

In [ ]:
sg.uo.dfprint(z)

In [ ]:
z.corr()

In [ ]:
from sklearn.ensemble import RandomForestClassifier as rfc
mod = rfc().fit(matrix, l)
pred = mod.predict(matrix)
from sklearn.metrics import accuracy_score
accuracy_score(l,pred)
ftscore = mod.feature_importances_

In [ ]:
import umap
e = umap.UMAP().fit_transform(matrix)
plt.scatter(*e.T,c=l, cmap='tab20')


In [ ]:
import yoda.ml.simpleMl as sml

def slowplot(X,y):
    z = np.unique(y)
    for g in lmz.Grouper(z,9):
        print(g)
        translate = {asd:i for i,asd in enumerate(g)}
        plt.scatter(*X.T, c= [ translate[l] if l in g else -1 for l in y  ], cmap = 'tab10')
        plt.show()
        plt.close()
slowplot(e,l)       

In [ ]:
import umap
from sklearn.decomposition import PCA
scr = ftscore[ftscore  > 0.00001]
mat = matrix.T[ftscore > 0.00001]
histomat = np.sort(mat, axis =0)
e = umap.UMAP(n_neighbors = 100).fit_transform(histomat)
# e = PCA(n_components=2).fit_transform(histomat)
plt.scatter(*e.T,c=scr)

In [ ]:
%%time
print(0)

In [ ]:
sml.knn_accuracy(m2,labels)

In [ ]:
lengths = [aa.alignment.shape[0] for aa in a]
z= [(aa.clusterlabel, aa.alignment.shape[0],i) for i,aa in enumerate(a)]
for v in sorted(z):
    print(v)

In [ ]:
ed.draw_graph(graphs[243], vertex_color = 'weight')
plt.show()
ed.draw_graph(graphs[226], vertex_color = 'weight')
plt.show()
ed.draw_graph(graphs[248], vertex_color = 'weight')
plt.show()

In [ ]:
my_array = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9])

# Define the value you want to search for
search_value = 4

# Use numpy.where() to get the indices where the condition is true
indices = np.where(my_array == search_value)[0]
indices